# Libraries

In [23]:
!python -V # --version

Python 3.12.1


In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

import pickle # To be able to save the model

In [25]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment') # Set experiment name. MLflow will create it if it doesn't exist and assigns all runs to it.

<Experiment: artifact_location='/workspaces/mlops-zoomcamp-2025/02-experiment-tracking/mlruns/1', creation_time=1748027473246, experiment_id='1', last_update_time=1748027473246, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
# Output for the first time you run this code:
# Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.
# <Experiment: artifact_location='/workspaces/mlops-zoomcamp-2025/02-experiment-tracking/mlruns/1', creation_time=1748027473246, experiment_id='1', last_update_time=1748027473246, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

# Output for when the experiment already exists:
# <Experiment: artifact_location='/workspaces/mlops-zoomcamp-2025/02-experiment-tracking/mlruns/1', creation_time=1748027473246, experiment_id='1', last_update_time=1748027473246, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>


In [26]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

# hyperopt - library that uses Bayesian optimisation to find the best hyperparameters
# fmin - method to minimise the objective function
# tpe - Tree-structured Parzen Estimator, a Bayesian optimisation algorithm
# hp - hyperopt library for defining the search space
# STATUS_OK - status of the trial
# Trials - class to keep track of the trials
# scope - used to define the search space for hyperparameters

# Data

Use the following commands to download the data locally: 
- `mkdir /workspaces/mlops-zoomcamp-2025/01-intro/data`
  
- `cd 01-intro/data`
  
- `wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet` - download data for January, 2021

- `wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet` - download data for February, 2021

In [27]:
%%time 

path_to_data = './data/green_tripdata_2021-01.parquet'
df = pd.read_parquet(path_to_data)

CPU times: user 27.1 ms, sys: 7.88 ms, total: 35 ms
Wall time: 16.6 ms


In [28]:
# Calculate target variable: trip duration
df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']

# Convert duration to minutes
df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

# Check percentage of data within 1 and 60 minute durations
trip_duration_1_to_60_condition = (df['duration'] >= 1) & (df['duration'] <= 60)

# Filter the data to only include trips with duration between 1 and 60 minutes
df = df.loc[trip_duration_1_to_60_condition]

# Feature Engineering

In [29]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [30]:
# Use one-hot encoding to convert categorical columns to numerical
# One-hot encoding converts strings into categorical variables. Convert int64 to string first

df[categorical] = df[categorical].astype(str)

In [31]:
%%time 

train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# Target variable
target = 'duration'
y_train = df[target].values

CPU times: user 273 ms, sys: 15 ms, total: 288 ms
Wall time: 287 ms


# Linear Regression

In [32]:
%%time 

lr = LinearRegression()
lr.fit(X_train, y_train)

# Make predictions
y_pred = lr.predict(X_train)

# Evaluate the performance of the model: use RMSE 
root_mean_squared_error(y_train, y_pred)

CPU times: user 143 ms, sys: 908 μs, total: 144 ms
Wall time: 22.2 ms


9.827368941909368

# Function to pre-process the data

In [33]:
def read_dataframe(filename):
    # Read in the data
    df = pd.read_parquet(filename)

    # Calculate target variable: trip duration
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']

    # Convert duration to minutes
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

    # Check percentage of data within 1 and 60 minute durations
    trip_duration_1_to_60_condition = (df['duration'] >= 1) & (df['duration'] <= 60)
    df = df.loc[trip_duration_1_to_60_condition]

    # Feature engineering
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [34]:
%%time 

df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

CPU times: user 477 ms, sys: 71.8 ms, total: 549 ms
Wall time: 512 ms


In [35]:
# Check the size of dataframes
len(df_train), len(df_val)

(73908, 61921)

In [36]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts) # Notice, that we only transform the validation set, not fitting required

In [37]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [38]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

10.496651120492572

# LASSO Regression

In [39]:
# Default parameters of Lasso
lr = Lasso()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

12.212583224318818

In [40]:
# Change default parameters of Lasso
lr = Lasso(alpha=0.001)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

10.481255832596219

In [41]:
%%time 

# Save the dictionary vectoriser and selected model
# mode wb - write binary
with open('./models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

CPU times: user 0 ns, sys: 1.51 ms, total: 1.51 ms
Wall time: 938 μs


# Keeping track of runs with MLflow

In [ ]:
with mlflow.start_run(): # Start a new run. Everything below will be associated with this run
    # Start logging information
    mlflow.set_tag('developer', 'timur')

    mlflow.log_param('train-data-path', './data/green_tripdata_2021-01.parquet') # Any parameters relevant to the model
    mlflow.log_param('valid-data-path', './data/green_tripdata_2021-02.parquet') 

    
    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)    
    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric('rmse', rmse)

    # Save the model as an artifact - maybe not the best way to do it, but this works
    mlflow.log_artifact(local_path='./models/lin_reg.bin', artifact_path='models_pickle')

# A different way to save the model

In [43]:
# To use XGBost, we need to convert the data into DMatrix format
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
# Start a new run
with mlflow.start_run():
    best_params = {
    "learning_rate": 0.14539214740033007,
    "max_depth": 11,
    "min_child_weight": 3.4888218672829265,
    "objective": "reg:linear",
    "reg_alpha": 0.012807793708352775,
    "reg_lambda": 0.25871874053728533,
    "seed": 42
    }

    mlflow.log_params(best_params) # Log the parameters

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse', rmse) # Log the metric

    # Log the model
    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')

# Save the model and the preprocessor

In [ ]:
# Start a new run
with mlflow.start_run():
    best_params = {
    "learning_rate": 0.14539214740033007,
    "max_depth": 11,
    "min_child_weight": 3.4888218672829265,
    "objective": "reg:linear",
    "reg_alpha": 0.012807793708352775,
    "reg_lambda": 0.25871874053728533,
    "seed": 42
    }

    mlflow.log_params(best_params) # Log the parameters

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse', rmse) # Log the metric

    # Save the preprocessor
    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')

    # Log the model
    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')

# Make predictions with saved model

In [47]:
# Use the uri (run id) of the logged model
logged_model = 'runs:/90a87b8981f449c3bfffe96d1b770877/models_mlflow' 

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [50]:
# Check the characteristics of the model - here the model is loaded a Python function
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 90a87b8981f449c3bfffe96d1b770877

In [52]:
# Load the model differently - as an XGBoost model
xgboost_model = mlflow.xgboost.load_model(model_uri=logged_model)

In [53]:
xgboost_model

In [54]:
# Make predictions with the loaded model
y_pred = xgboost_model.predict(valid)

In [55]:
y_pred[:10]

array([15.212646 ,  6.6534753, 16.98219  , 22.952501 ,  9.815957 ,
       17.562984 , 11.177028 ,  9.094591 ,  8.665553 , 17.606    ],
      dtype=float32)

# Hyperparameter Tuning with MLflow

In [26]:
# To use XGBost, we need to convert the data into DMatrix format
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [24]:
# Define the objective function
def objective(params):
    # params - set of hyperparameters for XGBoost

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
# Define the search space for hyperparameters
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # values between exp(-3) and exp(0), -0.05 and 1
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear', # reg:linear is now deprecated in favor of reg:squarederror
    'seed': 42
}

In [ ]:
# Pass the information about objective function and search space to fmin
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest, # Alogorithm to use for hyperparameter tuning
    max_evals=50,
    trials=Trials(),
    verbose=False # Set to True to see the progress of the trials
)

# Train the model with the best hyperparameters

In [ ]:
# Based on the results of the 50 runs, get the model with the lowest RMSE

params = {
    "learning_rate": 0.14539214740033007,
    "max_depth": 11,
    "min_child_weight": 3.4888218672829265,
    "objective": "reg:linear",
    "reg_alpha": 0.012807793708352775,
    "reg_lambda": 0.25871874053728533,
    "seed": 42
}

# Use autologging to log the parameters. This simplifies the process of logging parameters and metrics
mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

# Saving the Model

In [127]:
%%time 

# Save the dictionary vectoriser and selected model
# mode wb - write binary
with open('./models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

CPU times: user 7.09 ms, sys: 0 ns, total: 7.09 ms
Wall time: 15.2 ms
